## Step 2: Group movies by person, for each role

Because the process to analyze the data is so similar across the various roles we're interested in, I'm going to write a loop that does all the analyses iteratively for each role `ppl` (people) in `pplorder`.

This will use two dictionaries:

1. `ppldfdictpre` has key `ppl` for each item in `pplorder`, for value dataframe which results from the original `comp` composite dataframe exploded on the column named `ppl`. This will be used for downstream calculation of the career ROI% for each person, done by using the sums of the worldwide gross and budget numbers for all the movies each person has worked on, `ROI_careertot%`. This will also be used to get the number of movies each person (in role `ppl`) worked on, `movie_count`.
2. `ppldfdict` has key `ppl` for each item in `pplorder`, for value dataframe which results from the original `comp` composite dataframe, grouped by individual person id value (`nconst` in `namekey` dataframe) in column `ppl`, and the means for all the numerical columns of these groups. The resulting dataframe is trimmed to only include necessary columns for downstream analysis: `start_year`, `runtime_minutes`, `averagerating`, `numvotes`, `worldwide_gross_inf`, `production_budget_inf`, `ROI_careertot%`, and `movie_count`.

In [ ]:
# create dictionaries to store a dataframe (value) for each role (key)
ppldfdictpre = {} # initialize dictionary for explode/drop_duplicates step
ppldfdict = {}    # initialize dictionary to store end result

for ppl in pplorder: # loop over each role (`ppl`) in `pplorder`
    
    # for each role (`ppl`):

    # explode lists present in the column named `ppl` in composite dataframe (`comp`)
    # drop the duplicates ("director" & "writer" in particular was expected to have dupes)
        # (duplicates found by subset to avoid list error with other `ppl` columns in `comp`)
    # store the resulting dataframe as a value with key `ppl` in `ppldfdictpre` dictionary
    ppldfdictpre[ppl] = comp.explode(ppl).drop_duplicates(subset=['tconst','primary_title','start_year',ppl])
    
    # for key `ppl` in `ppldfdictpre` dictionary, take the value dataframe
    # group the `ppl` column by name id value (`nconst` in `namekey` dataframe)
    # take the mean of the aggregated groups' numerical columns
    # store the resulting dataframe as a value with key `ppl` in `ppldfdict` dictionary
    ppldfdict[ppl] = ppldfdictpre[ppl].groupby(ppl).mean()
    
    # for key `ppl` in `ppldfdictpre` dictionary, take the value dataframe
    # group the `ppl` column by name id value (`nconst` in `namekey` dataframe)
    # take the sum of the aggregated groups' numerical columns
    # do series math: 
        # use those summed columns to calculate ROI% with `worldwide_gross_inf` and `production_budget_inf`
        # '_inf' stands for "adjusted for inflation"
    # store the resulting series as column `ROI_careertot%` in `ppldfdict[ppl]`
    ppldfdict[ppl]['ROI_careertot%'] = (ppldfdictpre[ppl].groupby(ppl).sum()['worldwide_gross_inf'] -\
                                     ppldfdictpre[ppl].groupby(ppl).sum()['production_budget_inf']) /\
                                     ppldfdictpre[ppl].groupby(ppl).sum()['production_budget_inf']*100
    
    # for key `ppl` in `ppldfdictpre` dictionary, take the value dataframe
    # group the `ppl` column by name id value (`nconst` in `namekey` dataframe)
    # get the number of rows per aggregated group with .size()
    ppldfdict[ppl]['movie_count'] = ppldfdictpre[ppl].groupby(ppl).size()
    
    # for key `ppl` in `ppldfdict` dictionary, sort by `ROI_careertot%` descending
    ppldfdict[ppl]= ppldfdict[ppl].sort_values(by='ROI_careertot%',ascending=False)
    
    # for key `ppl` in `ppldfdict` dictionary, trim columns to only what we need downstream
    ppldfdict[ppl] = ppldfdict[ppl][['start_year','runtime_minutes','averagerating','numvotes','worldwide_gross_inf',\
                                     'production_budget_inf','ROI_careertot%','movie_count']]

## Step 3: Merge with name info

Now we'll use the `namekey` dataframe to yield the name info for each `nconst` value in the `ppl` role dataframes.

In [ ]:
# merge the role dataframes with the name info
ppl_comp = {} # dictionary for composite people dataframes
ppl_nums = {} # dictionary for counts of each dataframe
for ppl in ppldfdict.keys():
    ppl_comp[ppl] = ppldfdict[ppl].merge(namekey, how='left', left_on=ppl, right_on='nconst')
    ppl_nums[ppl] = len(ppl_comp[ppl]) # number of people in each role

## Step 4: Find average movie counts and %ROI for each dataframe
Create a new dictionary:
- `ppl_comp_avgs`

In [ ]:
ppl_comp_avgs = {}
for ppl in ppl_comp.keys():
    ppl_comp_avgs[f'{ppl}_movie_count'] = ppl_comp[ppl]['movie_count'].mean()
    ppl_comp_avgs[f'{ppl}_%ROI_avg'] = ppl_comp[ppl]['ROI_careertot%'].mean()